# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [31]:
from my_func import complete_cleaning
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

df_1 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv")
df_2 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv")
df_3 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
df_3 = df_3.rename(columns = {"State":"ST" , "Gender":"GENDER"},errors="raise")


In [35]:
df = pd.concat([df_1,df_2,df_3],)

In [43]:
df = complete_cleaning(df)

/home/tiago/Documents/GitHub/Labs/lab-dw-data-structuring-and-combining/my_func.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.customer_lifetime_value = df.customer_lifetime_value.fillna(0)
/home/tiago/Documents/GitHub/Labs/lab-dw-data-structuring-and-combining/my_func.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.gender = df.gender.fillna(df.gender.describe().top)


In [49]:
df.isnull().any()

customer                     False
state                        False
gender                       False
education                    False
customer_lifetime_value      False
income                       False
monthly_premium_auto         False
number_of_open_complaints    False
policy_type                  False
vehicle_class                False
total_claim_amount           False
dtype: bool

# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [179]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv")
df = df.drop("unnamed:_0", axis=1)
pivot_df = df.pivot_table(index ="sales_channel", values= "total_claim_amount", aggfunc='sum')
pivot_df.total_claim_amount = pivot_df.total_claim_amount.apply(lambda x : round(x,2))
pivot_df

,total_claim_amount
sales_channel,
Agent,1810226.82
Branch,1301204.00
Call Center,926600.82
Web,706600.04


2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [261]:
df_multiindex = df.set_index(["gender", "education"])
df_pivot2 =df_multiindex.pivot_table(index=["gender", "education"], values="customer_lifetime_value", aggfunc='sum')
df_pivot2.customer_lifetime_value = df_pivot2.customer_lifetime_value.apply(lambda x : round(x,2))
df_pivot2

customer_lifetime_value
gender education                                    
F      Bachelor                          13386258.11
       College                           12460107.91
       Doctor                             1465701.78
       High School or Below              13793600.12
       Master                             3874600.25
M      Bachelor                          12110061.83
       College                           12851725.02
       Doctor                             1490482.06
       High School or Below              12680914.19
       Master                             3365559.06

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [315]:
complains = {
    "date": ["15-05-2024", "15-05-2024", "13-05-2024", "13-05-2024", "14-05-2024"],
    "policyType": ["Car","Car","Health","Theft","Theft"],
    'complaints': [1,1,1,1,1]}

In [317]:
df = pd.DataFrame(complains)
df.date = pd.to_datetime(df.date, yearfirst=True)
df["month"] = df["date"].dt.to_period('M')

/tmp/ipykernel_49163/2114089593.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.date = pd.to_datetime(df.date, yearfirst=True)


In [319]:
df

,date,policyType,complaints,month
0,2024-05-15,Car,1,2024-05
1,2024-05-15,Car,1,2024-05
2,2024-05-13,Health,1,2024-05
3,2024-05-13,Theft,1,2024-05
4,2024-05-14,Theft,1,2024-05


In [321]:
df = df.groupby(["policyType","month"]).size().reset_index(name="complaints")

In [323]:
df

,policyType,month,complaints
0,Car,2024-05,2
1,Health,2024-05,1
2,Theft,2024-05,2
